# Import Rainfall from Historical Archive

Each CSV file in the Archive Folder is filtered just to keep the information about the rainfall stations defined below.

# Rain Dataset

In [2]:
import os
import json
import pandas as pd
import bng_latlon as bl
import numpy as np
import matplotlib.pyplot as plt

In [9]:
data_path = os.path.join(os.path.join("..", "data"))

dir_store_path = os.path.join(data_path, "temporary results")
dir_further_features_path = os.path.join(data_path, "further_features")
archive_folder = os.path.join(data_path, "historical_archive")
rainfall_folder = os.path.join(data_path, "rainfall")

In [4]:
rainfall_station_id_list = [
    "289102TP",
    "246424TP",
    "287283TP",
    "286392TP",
    "287946TP",
    "288092TP",
    "287864TP",
    "246847TP",
    "380002TP",
    "E8290",
    "239374TP",
    "239579TP",
    "290007TP",
    "540001RG",
    "238097TP",
    "245176TP",
    "291477TP",
    "E15500",
]

In [8]:
stations_dict = {}

for station in rainfall_station_id_list:
    stations_dict[station] = pd.DataFrame()

for file in os.listdir(archive_folder):
    df = pd.read_csv(
        os.path.join(archive_folder, file), parse_dates=["dateTime"]
    )
    
    for station in rainfall_station_id_list:
        filtered_df = df[df['measure'].str.contains(station)]
        stations_dict[station] = pd.concat([stations_dict[station], filtered_df])

/var/folders/z5/plf0_b5s39nb0_gctbxmpndc0000gn/T/ipykernel_39677/2058017314.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(
/var/folders/z5/plf0_b5s39nb0_gctbxmpndc0000gn/T/ipykernel_39677/2058017314.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(
/var/folders/z5/plf0_b5s39nb0_gctbxmpndc0000gn/T/ipykernel_39677/2058017314.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(
/var/folders/z5/plf0_b5s39nb0_gctbxmpndc0000gn/T/ipykernel_39677/2058017314.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(
/var/folders/z5/plf0_b5s39nb0_gctbxmpndc0000gn/T/ipykernel_39677/2058017314.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.

In [12]:
for station, df in stations_dict.items():
    # check if the path rainfall folder + station exists
    df.to_csv(os.path.join(rainfall_folder, f"{station}.csv"), index=False)

# Other

In [6]:
raw_rain_sites_df = pd.read_excel(
    os.path.join(dir_further_features_path, "Rainfall API Thames.xlsx"),
    sheet_name="Playing with data",
)
raw_deptford_df = pd.read_excel(
    os.path.join(dir_further_features_path, "Rainfall API Thames.xlsx"),
    sheet_name="Deptford",
)
raw_holland_park_df = pd.read_excel(
    os.path.join(dir_further_features_path, "Rainfall API Thames.xlsx"),
    sheet_name="Holland Park",
)
raw_putney_heath_df = pd.read_excel(
    os.path.join(dir_further_features_path, "Rainfall API Thames.xlsx"),
    sheet_name="Putney Heath",
)
raw_hogsmill_valley_df = pd.read_excel(
    os.path.join(dir_further_features_path, "Rainfall API Thames.xlsx"),
    sheet_name="Hogsmill Valley",
)

### Rain Sites

In [7]:
rain_sites_df = raw_rain_sites_df.copy()

# Date and Time adjustments
rain_sites_df["Latest Data"] = pd.to_datetime(rain_sites_df["Latest Data"])
rain_sites_df["Date"] = rain_sites_df["Latest Data"].dt.date
rain_sites_df["Time"] = rain_sites_df["Latest Data"].dt.time

date_col = rain_sites_df.pop("Date")
time_col = rain_sites_df.pop("Time")

rain_sites_df.insert(2, "Date", date_col)
rain_sites_df.insert(3, "Time", time_col)

rain_sites_df = rain_sites_df.drop(columns=["Latest Data"])

# Location adjustments
rain_sites_df["Lat"] = rain_sites_df["Location"].str.split(",").str[0]
rain_sites_df["Lon"] = rain_sites_df["Location"].str.split(",").str[1]

lat_col = rain_sites_df.pop("Lat")
lon_col = rain_sites_df.pop("Lon")

rain_sites_df.insert(4, "Lat", lat_col)
rain_sites_df.insert(5, "Lon", lon_col)

rain_sites_df = rain_sites_df.drop(columns=["Location"])

In [8]:
rain_sites_df

,Site,Date,Time,Lat,Lon,Rainfall (past 15 minutes)
0,Deptford,2023-11-17,06:15:00,51.476541,-0.017866,0
1,Holland Park,2023-11-17,08:30:00,51.501145,-0.205547,0
2,Putney Heath,2023-11-17,08:30:00,51.449283,-0.224851,0
3,Hogsmill Valley,2023-11-17,08:30:00,51.400709,-0.284238,0


### Deptford Site

In [9]:
deptford_df = raw_deptford_df.copy()

# Date and Time adjustments
deptford_df["DateTime"] = pd.to_datetime(deptford_df["DateTime"])
deptford_df["Date"] = deptford_df["DateTime"].dt.date
deptford_df["Time"] = deptford_df["DateTime"].dt.time

date_col = deptford_df.pop("Date")
time_col = deptford_df.pop("Time")

deptford_df.insert(2, "Date", date_col)
deptford_df.insert(3, "Time", time_col)

deptford_df = deptford_df.drop(columns=["DateTime"])

deptford_df.rename(columns={"Rainfall": "Rainfall (mm)"}, inplace=True)
deptford_df.drop(columns=["unit", "period_s"], inplace=True)

deptford_df

,Station_ID,Date,Time,Rainfall (mm)
0,289102TP,2023-03-11,18:00:00,0.0
1,289102TP,2023-03-11,18:15:00,0.0
2,289102TP,2023-03-11,18:30:00,0.2
3,289102TP,2023-03-11,18:45:00,0.0
4,289102TP,2023-03-11,19:00:00,0.0
...,...,...,...,...
22678,289102TP,2023-11-17,05:15:00,0.0
22679,289102TP,2023-11-17,05:30:00,0.0
22680,289102TP,2023-11-17,05:45:00,0.0
22681,289102TP,2023-11-17,06:00:00,0.0


### Holland Park Site

In [10]:
holland_park_df = raw_holland_park_df.copy()

# Date and Time adjustments
holland_park_df["DateTime"] = pd.to_datetime(holland_park_df["DateTime"])
holland_park_df["Date"] = holland_park_df["DateTime"].dt.date
holland_park_df["Time"] = holland_park_df["DateTime"].dt.time

date_col = holland_park_df.pop("Date")
time_col = holland_park_df.pop("Time")

holland_park_df.insert(2, "Date", date_col)
holland_park_df.insert(3, "Time", time_col)

holland_park_df = holland_park_df.drop(columns=["DateTime"])

holland_park_df.rename(columns={"Rainfall": "Rainfall (mm)"}, inplace=True)
holland_park_df.drop(columns=["unit", "period_s"], inplace=True)

holland_park_df

,Station_ID,Date,Time,Rainfall (mm)
0,246424TP,2023-03-11,18:15:00,0.0
1,246424TP,2023-03-11,18:30:00,0.0
2,246424TP,2023-03-11,19:00:00,0.4
3,246424TP,2023-03-11,19:15:00,0.4
4,246424TP,2023-03-11,19:30:00,0.0
...,...,...,...,...
22433,246424TP,2023-11-17,07:30:00,0.0
22434,246424TP,2023-11-17,07:45:00,0.0
22435,246424TP,2023-11-17,08:00:00,0.0
22436,246424TP,2023-11-17,08:15:00,0.0


### Putney Heath Site

In [11]:
putney_heath_df = raw_putney_heath_df.copy()

# Date and Time adjustments
putney_heath_df["DateTime"] = pd.to_datetime(putney_heath_df["DateTime"])
putney_heath_df["Date"] = putney_heath_df["DateTime"].dt.date
putney_heath_df["Time"] = putney_heath_df["DateTime"].dt.time

date_col = putney_heath_df.pop("Date")
time_col = putney_heath_df.pop("Time")

putney_heath_df.insert(2, "Date", date_col)
putney_heath_df.insert(3, "Time", time_col)

putney_heath_df = putney_heath_df.drop(columns=["DateTime"])

putney_heath_df.rename(columns={"Rainfall": "Rainfall (mm)"}, inplace=True)

putney_heath_df.drop(columns=["unit", "period_s"], inplace=True)

putney_heath_df

,Station_ID,Date,Time,Rainfall (mm)
0,287283TP,2023-03-11,18:30:00,0.1
1,287283TP,2023-03-11,18:45:00,0.0
2,287283TP,2023-03-11,19:00:00,0.0
3,287283TP,2023-03-11,19:15:00,0.2
4,287283TP,2023-03-11,19:30:00,0.0
...,...,...,...,...
22840,287283TP,2023-11-17,07:30:00,0.0
22841,287283TP,2023-11-17,07:45:00,0.0
22842,287283TP,2023-11-17,08:00:00,0.0
22843,287283TP,2023-11-17,08:15:00,0.0


### Hogsmill Valley Site

In [12]:
hogsmill_valley_df = raw_hogsmill_valley_df.copy()

# Date and Time adjustments
hogsmill_valley_df["DateTime"] = pd.to_datetime(hogsmill_valley_df["DateTime"])
hogsmill_valley_df["Date"] = hogsmill_valley_df["DateTime"].dt.date
hogsmill_valley_df["Time"] = hogsmill_valley_df["DateTime"].dt.time

date_col = hogsmill_valley_df.pop("Date")
time_col = hogsmill_valley_df.pop("Time")

hogsmill_valley_df.insert(2, "Date", date_col)
hogsmill_valley_df.insert(3, "Time", time_col)

hogsmill_valley_df = hogsmill_valley_df.drop(columns=["DateTime"])

hogsmill_valley_df.rename(columns={"Rainfall": "Rainfall (mm)"}, inplace=True)

hogsmill_valley_df.drop(columns=["unit", "period_s"], inplace=True)

hogsmill_valley_df

,Station_ID,Date,Time,Rainfall (mm)
0,286392TP,2023-03-11,18:30:00,0.0
1,286392TP,2023-03-11,18:45:00,0.0
2,286392TP,2023-03-11,19:00:00,0.0
3,286392TP,2023-03-11,19:15:00,0.1
4,286392TP,2023-03-11,19:30:00,0.1
...,...,...,...,...
22839,286392TP,2023-11-17,07:30:00,0.0
22840,286392TP,2023-11-17,07:45:00,0.0
22841,286392TP,2023-11-17,08:00:00,0.0
22842,286392TP,2023-11-17,08:15:00,0.0


## Merge Data

In [13]:
deptford_df.insert(0, "Site", "Deptford")
holland_park_df.insert(0, "Site", "Holland Park")
putney_heath_df.insert(0, "Site", "Putney Heath")
hogsmill_valley_df.insert(0, "Site", "Hogsmill Valley")

In [14]:
deptford_df.insert(
    2,
    "Lat",
    rain_sites_df.loc[rain_sites_df["Site"] == "Deptford", "Lat"].values[0],
)
deptford_df.insert(
    3,
    "Lon",
    rain_sites_df.loc[rain_sites_df["Site"] == "Deptford", "Lon"].values[0],
)

holland_park_df.insert(
    2,
    "Lat",
    rain_sites_df.loc[rain_sites_df["Site"] == "Holland Park", "Lat"].values[0],
)
holland_park_df.insert(
    3,
    "Lon",
    rain_sites_df.loc[rain_sites_df["Site"] == "Holland Park", "Lon"].values[0],
)

putney_heath_df.insert(
    2,
    "Lat",
    rain_sites_df.loc[rain_sites_df["Site"] == "Putney Heath", "Lat"].values[0],
)
putney_heath_df.insert(
    3,
    "Lon",
    rain_sites_df.loc[rain_sites_df["Site"] == "Putney Heath", "Lon"].values[0],
)

hogsmill_valley_df.insert(
    2,
    "Lat",
    rain_sites_df.loc[rain_sites_df["Site"] == "Hogsmill Valley", "Lat"].values[
        0
    ],
)
hogsmill_valley_df.insert(
    3,
    "Lon",
    rain_sites_df.loc[rain_sites_df["Site"] == "Hogsmill Valley", "Lon"].values[
        0
    ],
)

In [15]:
full_df = pd.concat(
    [deptford_df, holland_park_df, putney_heath_df, hogsmill_valley_df]
).reset_index(drop=True)

full_df

,Site,Station_ID,Lat,Lon,Date,Time,Rainfall (mm)
0,Deptford,289102TP,51.476541,-0.017866,2023-03-11,18:00:00,0.0
1,Deptford,289102TP,51.476541,-0.017866,2023-03-11,18:15:00,0.0
2,Deptford,289102TP,51.476541,-0.017866,2023-03-11,18:30:00,0.2
3,Deptford,289102TP,51.476541,-0.017866,2023-03-11,18:45:00,0.0
4,Deptford,289102TP,51.476541,-0.017866,2023-03-11,19:00:00,0.0
...,...,...,...,...,...,...,...
90805,Hogsmill Valley,286392TP,51.400709,-0.284238,2023-11-17,07:30:00,0.0
90806,Hogsmill Valley,286392TP,51.400709,-0.284238,2023-11-17,07:45:00,0.0
90807,Hogsmill Valley,286392TP,51.400709,-0.284238,2023-11-17,08:00:00,0.0
90808,Hogsmill Valley,286392TP,51.400709,-0.284238,2023-11-17,08:15:00,0.0


In [16]:
# Combine Date and Time columns
full_df["DateTime"] = pd.to_datetime(
    full_df["Date"].astype(str) + " " + full_df["Time"].astype(str)
)

In [18]:
full_df.drop(columns=["Date", "Time"], inplace=True)

In [19]:
full_df.to_excel(
    os.path.join(dir_store_path, "rainfall_dataset.xlsx"), index=False
)